In [112]:
import os
import json
import pickle
import pydicom
from PIL import Image, ImageDraw, ImageEnhance

In [113]:
def list_folders(directory):
    return [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]

folders = list_folders('Data_unzip')
print(folders)

['case7048', 'case7060', 'case7074', 'case7128', 'case7114', 'case7100', 'case1029', 'case1001', 'case1015', 'case1028', 'case7101', 'case7115', 'case7129', 'case7075', 'case7061', 'case7049', 'case7077', 'case7063', 'case7088', 'case7103', 'case7117', 'case1016', 'case1002', 'case1003', 'case1017', 'case7116', 'case7102', 'case7089', 'case7062', 'case7076', 'case7072', 'case7066', 'case7099', 'case7106', 'case7112', 'case1013', 'case1007', 'case1006', 'case1012', 'case7113', 'case7107', 'case7098', 'case7067', 'case7073', 'case7065', 'case7071', 'case7059', 'case7111', 'case7105', 'case7139', 'case1004', 'case1010', 'case1038', 'case1039', 'case1011', 'case1005', 'case7138', 'case7104', 'case7110', 'case7058', 'case7070', 'case7064', 'case7003', 'case7017', 'case7163', 'case7162', 'case7016', 'case7002', 'case7028', 'case7014', 'case7148', 'case7160', 'case1049', 'case1048', 'case7161', 'case7149', 'case7001', 'case7015', 'case7029', 'case7011', 'case7005', 'case7039', 'case7165', 'ca

In [114]:
p= 0
for case in folders:
    slics = list_folders("Data_unzip/" + str(case)) 
    p2 = 0
    for slic in slics:
        path_stem  = "Data_unzip/"+ str(case) +"/" + str(slic)
        p = p+1
        p2 = p2+1
    print(str(case) + " : " + str(p2))
print(p)

case7048 : 5
case7060 : 5
case7074 : 5
case7128 : 5
case7114 : 5
case7100 : 5
case1029 : 5
case1001 : 5
case1015 : 5
case1028 : 5
case7101 : 5
case7115 : 5
case7129 : 5
case7075 : 5
case7061 : 5
case7049 : 5
case7077 : 5
case7063 : 5
case7088 : 5
case7103 : 5
case7117 : 5
case1016 : 5
case1002 : 5
case1003 : 5
case1017 : 5
case7116 : 5
case7102 : 5
case7089 : 5
case7062 : 5
case7076 : 5
case7072 : 5
case7066 : 5
case7099 : 5
case7106 : 5
case7112 : 5
case1013 : 5
case1007 : 5
case1006 : 5
case1012 : 5
case7113 : 5
case7107 : 5
case7098 : 5
case7067 : 5
case7073 : 5
case7065 : 5
case7071 : 5
case7059 : 5
case7111 : 5
case7105 : 5
case7139 : 5
case1004 : 5
case1010 : 5
case1038 : 5
case1039 : 5
case1011 : 5
case1005 : 5
case7138 : 5
case7104 : 5
case7110 : 5
case7058 : 5
case7070 : 5
case7064 : 5
case7003 : 5
case7017 : 5
case7163 : 5
case7162 : 5
case7016 : 5
case7002 : 5
case7028 : 5
case7014 : 5
case7148 : 5
case7160 : 5
case1049 : 5
case1048 : 5
case7161 : 5
case7149 : 5
case7001 : 5

In [115]:
for case in folders:
    slics = list_folders("Data_unzip/" + str(case))  
#     slics2 = [int(slics[n][-4:]) for n in range(len(slics)) ]
#     slics2 = [slics[n] - min(slics) for n in range(len(slics)) ]
    
    #check
#     print(case)
#     print(slics)
#     print(slics2)

    for n in range(len(slics)):
        path_stem  = "Data_unzip/"+ str(case) +"/" + str(slics[n])
        path_json  = path_stem + "/AnnotationFile.Json"
        path_dicom = path_stem + "/DicomFile.dcm"
        path_dicom_ano = path_stem + "/DicomImage.jpg"

        ### Json block
        miss_annotation = 'Data_unzip/case7153/1.2.392.200036.9142.10002202.1020330994.3.20111202162010.49103'
        if path_stem != miss_annotation:
            with open(path_json, 'r', encoding='utf-8') as f:
                data = json.load(f)
            cg_points_lists = [item['cgPoints'] for item in data]
            CCC_ground_truth = len(cg_points_lists)
            simplified_data = [[[point['x'], point['y']] for point in sublist] for sublist in cg_points_lists]
        elif path_stem == miss_annotation:
            with open('Data_unzip/case7153/1.2.392.200036.9142.10002202.1020330994.3.20111202162010.49103/AnnotationFile.pkl', 'rb') as f:
                simplified_data = pickle.load(f)
            CCC_ground_truth = 1

        ### Dicom block
        dicom = pydicom.dcmread(path_dicom)
        dicom_array = dicom.pixel_array
        dicom_array_normalized = (dicom_array / dicom_array.max()) * 255
        dicom_image = Image.fromarray(dicom_array_normalized.astype('uint8'))

        ### Json+Dicom=Mask block
        mask = Image.new('L', (dicom.Columns, dicom.Rows), 0)
        draw = ImageDraw.Draw(mask)
        for coordinates in simplified_data:
            int_coordinates = [(int(x), int(y)) for x, y in coordinates]
            draw.polygon(int_coordinates, fill=255)

        enhancer = ImageEnhance.Brightness(mask)
        mask_brightened = enhancer.enhance(1)  # 透明度を調整 (0.5は半透明)
        composite = Image.composite(mask_brightened, dicom_image, mask_brightened)

        ### jpg block
        another_image = Image.open(path_dicom_ano).convert('RGB').resize(dicom_image.size)

        ### integration block
        canvas_width = dicom_image.width * 3
        canvas_height = dicom_image.height
        canvas = Image.new('RGB', (canvas_width, canvas_height), (255, 255, 255))
        canvas.paste(dicom_image, (0, 0))
        canvas.paste(composite, (dicom_image.width, 0))
        canvas.paste(another_image, (dicom_image.width * 2, 0))
#         canvas.show()

        ### Save block
        import os
        path_folder = "Data_base/" + str(case[4:8]) +"/"+ str(n)
        if not os.path.exists(path_folder):
            os.makedirs(path_folder)
            print(f"'{path_folder}' has been created.")
        else:
            print(f"'{path_folder}' already exists.")

        dicom.save_as(path_folder +"/data_dicom.dcm")
        mask.save(path_folder +"/data_mask.jpg")
        canvas.save(path_folder +"/data_check.jpg")
        with open(path_folder +"/data_CCC_ground_truth.json", "w") as f:
            json.dump(CCC_ground_truth, f)

'Data_base/7048/0' already exists.
'Data_base/7048/1' already exists.
'Data_base/7048/2' already exists.
'Data_base/7048/3' already exists.
'Data_base/7048/4' already exists.
'Data_base/7060/0' already exists.
'Data_base/7060/1' already exists.
'Data_base/7060/2' already exists.
'Data_base/7060/3' already exists.
'Data_base/7060/4' already exists.
'Data_base/7074/0' already exists.
'Data_base/7074/1' already exists.
'Data_base/7074/2' already exists.
'Data_base/7074/3' already exists.
'Data_base/7074/4' already exists.
'Data_base/7128/0' already exists.
'Data_base/7128/1' already exists.
'Data_base/7128/2' already exists.
'Data_base/7128/3' already exists.
'Data_base/7128/4' already exists.
'Data_base/7114/0' already exists.
'Data_base/7114/1' already exists.
'Data_base/7114/2' already exists.
'Data_base/7114/3' already exists.
'Data_base/7114/4' already exists.
'Data_base/7100/0' already exists.
'Data_base/7100/1' already exists.
'Data_base/7100/2' already exists.
'Data_base/7100/3' a